In [2]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kushagra3204/wheat-plant-diseases")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/wheat-plant-diseases


In [4]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.optimizers import Adam 
from sklearn.metrics import classification_report

2025-12-05 20:29:15.646904: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764966556.070141      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764966556.203486      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [5]:
BASE_PATH = '/kaggle/input/wheat-plant-diseases/wheat-plant-diseases/'

DATA_DIR_TRAIN = '/kaggle/input/wheat-plant-diseases/data/train'
DATA_DIR_TEST = '/kaggle/input/wheat-plant-diseases/data/test'
IMAGE_SIZE = (224, 224) 
BATCH_SIZE = 32
EPOCHS = 15
NUM_CLASSES = 5

In [6]:
TRAIN_CLASS_FOLDERS = ['Aphid', 'Blast', 'Mildew', 'Smut', 'Tan spot']
TEST_CLASS_FOLDERS = ['aphid_test', 'blast_test', 'mildew_test', 'smut_test', 'tan_spot_test']
REPORT_TARGET_CLASSES = ['Aphid', 'Wheat Blast', 'Powdery Mildew', 'Smut', 'Spot Blotch']

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=20, 
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

print("--- Initializing Data Generators ---")

--- Initializing Data Generators ---


In [8]:
# TRAINING GENERATOR
try:
    train_generator = train_datagen.flow_from_directory(
        DATA_DIR_TRAIN, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, 
        classes=TRAIN_CLASS_FOLDERS, class_mode='categorical', subset='training', shuffle=True
    )
    # VALIDATION GENERATOR
    validation_generator = train_datagen.flow_from_directory(
        DATA_DIR_TRAIN, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, 
        classes=TRAIN_CLASS_FOLDERS, class_mode='categorical', subset='validation', shuffle=False
    )
    # TEST GENERATOR
    test_generator = test_datagen.flow_from_directory(
        DATA_DIR_TEST, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, 
        classes=TEST_CLASS_FOLDERS, class_mode='categorical', shuffle=False
    )
    
    print(f"Training Samples: {train_generator.n}")
    print(f"Validation Samples: {validation_generator.n}")
    print(f"Test Samples: {test_generator.n}")

except Exception as e:
    print(f"\nERROR: Could not load data generators. Check the paths and folder names.")
    print(f"Details: {e}")
    # এখানে ValueError আসার সম্ভাবনা কম, যদি Path ও Folder Naming ঠিক থাকে
    exit()

Found 3770 images belonging to 5 classes.
Found 941 images belonging to 5 classes.
Found 250 images belonging to 5 classes.
Training Samples: 3770
Validation Samples: 941
Test Samples: 250


In [9]:
print("\n--- Building and Training VGG19 Model ---")

# 1. Load Base Model (VGG19)
base_model_vgg19 = VGG19(
    weights='imagenet', 
    include_top=False, 
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
)

for layer in base_model_vgg19.layers:
    layer.trainable = True

for layer in base_model_vgg19.layers[:10]:
    layer.trainable = False
    
x = base_model_vgg19.output
x = GlobalAveragePooling2D()(x) 
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model_vgg19.input, outputs=predictions)


--- Building and Training VGG19 Model ---


I0000 00:00:1764966582.400057      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1764966582.400652      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
# 3. Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.00001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv4 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv4 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv4 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 20,289,605 (77.40 MB)

 Trainable params: 18,554,117 (70.78 MB)

 Non-trainable params: 1,735,488 (6.62 MB)

In [11]:
# 4. Train Model
start_time = time.time()
history = model.fit(
    train_generator, 
    steps_per_epoch=train_generator.samples // BATCH_SIZE, 
    epochs=EPOCHS, 
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    verbose=1
)
training_time = time.time() - start_time



/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15


I0000 00:00:1764966591.450878     133 service.cc:148] XLA service 0x7b4f6c015930 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764966591.452453     133 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1764966591.452473     133 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1764966592.176683     133 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1764966613.382588     133 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


117/117 ━━━━━━━━━━━━━━━━━━━━ 166s 1s/step - accuracy: 0.3326 - loss: 1.5283 - val_accuracy: 0.6239 - val_loss: 0.9686
Epoch 2/15
  1/117 ━━━━━━━━━━━━━━━━━━━━ 41s 361ms/step - accuracy: 0.6875 - loss: 0.7803

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


117/117 ━━━━━━━━━━━━━━━━━━━━ 17s 145ms/step - accuracy: 0.6875 - loss: 0.7803 - val_accuracy: 0.6121 - val_loss: 0.9733
Epoch 3/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 88s 748ms/step - accuracy: 0.6408 - loss: 0.9226 - val_accuracy: 0.6325 - val_loss: 0.8812
Epoch 4/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 17s 147ms/step - accuracy: 0.5938 - loss: 1.0366 - val_accuracy: 0.6433 - val_loss: 0.8898
Epoch 5/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 85s 726ms/step - accuracy: 0.7017 - loss: 0.7805 - val_accuracy: 0.6616 - val_loss: 0.8385
Epoch 6/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 18s 148ms/step - accuracy: 0.6875 - loss: 0.8080 - val_accuracy: 0.6800 - val_loss: 0.8399
Epoch 7/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 86s 732ms/step - accuracy: 0.7691 - loss: 0.6330 - val_accuracy: 0.7220 - val_loss: 0.6795
Epoch 8/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 17s 146ms/step - accuracy: 0.6875 - loss: 0.6382 - val_accuracy: 0.7058 - val_loss: 0.6889
Epoch 9/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 86s 736ms/step - accuracy: 0.7895 - loss: 0.5884 - val

In [20]:
# 5. Evaluate and Report
print(f"\n--- VGG19 FINAL RESULTS ---")

# Evaluate on Test Generator
loss, accuracy = model.evaluate(test_generator, steps=test_generator.n // BATCH_SIZE + 1, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Training Time: {training_time:.2f} seconds")


--- VGG19 FINAL RESULTS ---
Test Accuracy: 0.8240
Training Time: 890.38 seconds


In [23]:
# Generate Classification Report
test_generator.reset()
Y_pred = model.predict(test_generator, steps=test_generator.n // BATCH_SIZE + 1, verbose=0)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

report = classification_report(
    y_true, 
    y_pred, 
    target_names=REPORT_TARGET_CLASSES, 
    zero_division=0,
    output_dict=True 
)

print("\n--- CLASSIFICATION REPORT ---")
print(classification_report(y_true, y_pred, target_names=REPORT_TARGET_CLASSES, zero_division=0))


--- CLASSIFICATION REPORT ---
                precision    recall  f1-score   support

         Aphid       0.78      0.78      0.78        50
   Wheat Blast       0.79      0.98      0.87        50
Powdery Mildew       0.93      0.78      0.85        50
          Smut       0.90      0.88      0.89        50
   Spot Blotch       0.74      0.70      0.72        50

      accuracy                           0.82       250
     macro avg       0.83      0.82      0.82       250
  weighted avg       0.83      0.82      0.82       250



In [24]:
model_name = "VGG19"

metrics_summary = {
    'Model': model_name,
    'Accuracy': report['accuracy'],
    'Precision': report['weighted avg']['precision'], 
    'Recall': report['weighted avg']['recall'],
    'F1-Score': report['weighted avg']['f1-score'],
    'Training Time (s)': training_time,
}

df_metrics = pd.DataFrame([metrics_summary])


In [26]:

results_file = 'model_performance_summary.csv'
if not os.path.exists(results_file):
    df_metrics.to_csv(results_file, index=False)
else:
    df_metrics.to_csv(results_file, mode='a', header=False, index=False)
    
print(f"\nMetrics saved to {results_file}")


Metrics saved to model_performance_summary.csv
